<a href="https://colab.research.google.com/github/HromovAndrey/II_Homework_mood_3_part_5/blob/main/notebook65ee529160.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import json
import pandas as pd

def load_df(csv_path='/kaggle/input/ga-customer-revenue-prediction/train_v2.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(csv_path,
                     converters={column: json.loads for column in JSON_COLUMNS},
                     dtype={'fullVisitorId': 'str'},
                     nrows=nrows)

    for column in JSON_COLUMNS:
        column_as_df = pd.json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    df.drop(columns=['customDimensions', 'hits'], inplace=True)

    df = df.fillna({'totals_pageviews':0,
                    'totals_transactionRevenue':0,
                    'totals_pageviews':0})

    df['device_isMobile'] = df['device_isMobile'].astype(int)
    df['totals_transactionRevenue'] = df['totals_transactionRevenue'].astype(int)

    cat_col = df.select_dtypes(include='object').columns

    df[cat_col] = df[cat_col].astype(str)

    return df

df = load_df('/kaggle/input/ga-customer-revenue-prediction/train_v2.csv', nrows=100_000)


df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/ga-customer-revenue-prediction/train_v2.csv'

In [ ]:
print(df.describe())
print(df.isnull().sum())

In [ ]:
from sklearn.model_selection import train_test_split

y = df['totals_transactionRevenue']
X = df.drop(columns='totals_transactionRevenue')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [ ]:
from sklearn.model_selection import train_test_split

y = df['totals_transactionRevenue']
X = df.drop(columns='totals_transactionRevenue')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

print(X_train_preprocessed.shape, X_test_preprocessed.shape)


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# Налаштування параметрів для пошуку
param_grid = {
    'num_leaves': [31, 50],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200]
}

lgb_model = lgb.LGBMRegressor()

grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train_preprocessed, y_train)

best_params = grid_search.best_params_
print(f"Найкращі параметри: {best_params}")

# Навчання моделі з найкращими параметрами
best_model = grid_search.best_estimator_

y_train_pred = best_model.predict(X_train_preprocessed)
y_test_pred = best_model.predict(X_test_preprocessed)

def get_metrics(y_true, y_pred):
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "R2": r2_score(y_true, y_pred)
    }

train_metrics = get_metrics(y_train, y_train_pred)
test_metrics = get_metrics(y_test, y_test_pred)

print("Train metrics:", train_metrics)
print("Test metrics:", test_metrics)

import matplotlib.pyplot as plt

metrics_df = pd.DataFrame({'Train': list(train_metrics.values()), 'Test': list(test_metrics.values())}, index=train_metrics.keys())
metrics_df.plot(kind='bar')
plt.title("Metrics on Train and Test Data")
plt.ylabel("Metric Value")
plt.show()


In [ ]:
import joblib


final_model = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', best_model)])

joblib.dump(final_model, 'gstore_revenue_model.pkl')


loaded_model = joblib.load('gstore_revenue_model.pkl')

example_data = X_test.iloc[:5, :]
example_pred = loaded_model.predict(example_data)

print("Predictions:", example_pred)
print("Actual revenues:", y_test.iloc[:5].values)
